In [1]:
import requests
import pandas as pd
from datetime import datetime

In [2]:
category_mapping = {
    7: "Квартира",
    8: "Дом",
    10: "Для бизнеса",
    11: "Земельный участок"
}

state_mapping = {
    False: "Новострой",
    True: "Вторичка",
}

def get_json_data(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: Unable to fetch data from {api_url}")
        return None

def create_dataframe(json_data):
    if json_data and 'results' in json_data:
        results = json_data['results']
        if not results:
            print("No results found. Skipping to the next iteration.")
            return pd.DataFrame()

        apartments = []
        for result in results:
            apartment = {
                "id": result.get("id"),
                "categoryId": category_mapping.get(result.get("categoryId")),
                "description": result.get("description"),
                "price": result.get("price"),
                "priceCurrency": result.get("priceCurrency"),
                "address": result.get("address"),
                "region": result["region"]["name"]["en"] if result.get("region") else None,
                "district": result["district"]["name"]["en"] if result.get("district") else None,
                "street": result["street"]["name"]["en"] if result.get("street") else None,
                "zone": result["zone"]["name"]["en"] if result.get("zone") else None,
                "room": result.get("room"),
                "square": result.get("square"),
                "floor": result.get("floor"),
                "floorTotal": result.get("floorTotal"),
                "isNewBuilding": state_mapping.get(result.get("isNewBuilding")),
                "repair": result.get("repair"),
                "foundation": result.get("foundation"),
                "residentialComplex": result["residentialComplex"]["name"]["en"] if result.get("residentialComplex") else None,
                "lat": result.get("lat"),
                "lng": result.get("lng"),
                "createdAt": result.get("createdAt"),
                "updatedAt": result.get("updatedAt"),
                "isUrgently": result.get("isUrgently"),
                "isVip": result.get("isVip"),
                "isPremium": result.get("isPremium"),
                "isFavorite": result.get("isFavorite"),
                "views": result.get("views"),
                "clicks": result.get("clicks"),
                "favorites": result.get("favorites")
            }
            apartments.append(apartment)
        return pd.DataFrame(apartments)
    else:
        print("Invalid JSON data. Skipping to the next iteration.")
        return pd.DataFrame()

def fetch_data(start_page, end_page):
    all_data = []

    for page_number in range(start_page, end_page):
        # Construct the API URL for the current page number and page
        api_url = f"{url_template}{page_number}"
        # Fetch JSON data
        json_data = get_json_data(api_url)

        # Check if the response is empty or results are empty
        if not json_data or not json_data.get('results'):
            print(f"No results for page={page_number}. Exiting the loop.")
            break

        # Create DataFrame
        df = create_dataframe(json_data)

        # Append DataFrame to the list
        all_data.append(df)
    
    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(all_data, ignore_index=True)
    return final_df

# Example for room__in=1
url_template = "https://api.uybor.uz/api/v1/listings?mode=search&includeFeatured=true&limit=20&embed=category%2CsubCategory%2CresidentialComplex%2Cregion%2Ccity%2Cdistrict%2Czone%2Cstreet%2Cmetro%2Cmedia%2Cuser%2Cuser.avatar%2Cuser.organization%2Cuser.organization.logo&order=upAt&operationType__eq=sale&priceCurrency__eq=usd&page="

start_page = 1
end_page = 400

final_df = fetch_data(start_page, end_page)

No results for page=242. Exiting the loop.


In [3]:
final_df = final_df.drop(["id", 'isVip', 'isPremium', 'isFavorite', 'views', 'clicks', 'favorites', 'street', 'zone', 'residentialComplex'], axis=1)
final_df = final_df.rename(columns={"floor": "Этаж", "floorTotal": "Этажность", "repair": "Ремонт",
                                    "foundation": "Материал", 'room': "Количество комнат", "square": "Площадь", 'region': 'Регион',
                                    'description': 'Название', 'price': 'Цена', 'priceCurrency': 'Валюта',
                                    'district': "Район", 'address': 'Адрес', 'isNewBuilding': 'Новое здание',
                                    'isUrgently': 'Срочно', "createdAt": "Дата публикации", "updatedAt": "Обновлён",
                                    "lat": "Широта", "lng": "Долгота", "categoryId": "Тип", "isNewBuilding": "Тип постройки"})

final_df['Дата публикации'] = pd.to_datetime(final_df['Дата публикации'], utc=True)
final_df['Обновлён'] = pd.to_datetime(final_df['Обновлён'], utc=True)
final_df['Валюта'] = final_df['Валюта'].str.upper()
repair_mapping = {
    'evro': 'Евроремонт',
    'chernovaya': 'Черновая отделка',
    'custom': 'Индивидуальный',
    'kapital': 'Капитальный',
    'sredniy': 'Средний',
    None: None  # Keep None values unchanged
}
material_mapping = {
    'kirpich': 'Кирпич',
    'monolit': 'Монолит',
    'panel': 'Панель',
    'other': 'Другой',
    None: None  # Keep None values unchanged
}
final_df['Ремонт'] = final_df['Ремонт'].replace(repair_mapping)
final_df['Материал'] = final_df['Материал'].replace(material_mapping)
final_df['Дата публикации'] = final_df['Дата публикации'].dt.strftime('%d.%m.%Y')
final_df['Обновлён'] = final_df['Обновлён'].dt.strftime('%d.%m.%Y')

In [4]:
columns_to_check = ["Источник", "Название", "Тип","Санузел", "Тип постройки", "Материал", "Широта", 
                    "Долгота", "Район", "Этаж", "Этажность", "Ремонт", "Площадь", 
                    "Количество комнат", "Дата публикации", "Валюта", "Цена", "Дата создания"]

# Create a new DataFrame with the specified columns
new_df = pd.DataFrame(columns=columns_to_check)

# Check if columns exist in final_df and create them with None values if not
for column in columns_to_check:
    if column not in final_df.columns:
        final_df[column] = None
        new_df[column] = None
    else:
        new_df[column] = final_df[column]

 
new_df["Источник"] = 'Uybor'
new_df["Район"] = ''
new_df["Дата создания"] = datetime.now().strftime("%d.%m.%Y")
new_df[columns_to_check]

,Источник,Название,Тип,Санузел,Тип постройки,Материал,Широта,Долгота,Район,Этаж,Этажность,Ремонт,Площадь,Количество комнат,Дата публикации,Валюта,Цена,Дата создания
0,Uybor,"\nПродается 100% доля МЧЖ ""TURKUAZ""\nЦена 8 37...",Для бизнеса,None,None,Кирпич,41.293129,69.287468,,NaN,3.0,Евроремонт,1580.0,None,26.02.2024,USD,650000,26.04.2024
1,Uybor,Сотилади !\n4/4/18 \nХолати каробка \nОб 93кв\...,Квартира,None,Вторичка,Монолит,41.332088,69.324409,,4.0,10.0,Черновая отделка,93.0,4,30.03.2024,USD,105000,26.04.2024
2,Uybor,Юнусабад 17. 1в2/4/5.\n38м2.\nСреднее состояни...,Квартира,None,Новострой,Панель,41.366066,69.310387,,4.0,5.0,Средний,38.0,1,27.10.2023,USD,46500,26.04.2024
3,Uybor,"Газалкентское водохранилище 2 дома 29,5 соток,...",Дом,None,None,Кирпич,41.570671,69.773605,,NaN,2.0,Евроремонт,800.0,6+,11.04.2024,USD,850000,26.04.2024
4,Uybor,"Шумиловский городок, мечеть Ислом Ота 2 дома 8...",Дом,None,None,Кирпич,41.279518,69.309570,,NaN,1.0,Средний,300.0,6+,10.04.2024,USD,340000,26.04.2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6004,Uybor,Идеальный вариант для Вашего бизнеса. 3.18 в э...,Земельный участок,None,None,None,41.258312,69.209473,,NaN,NaN,None,NaN,None,25.02.2024,USD,185000,26.04.2024
6005,Uybor,"Чиланзар 3, до метро 10 минут. Состояние хорош...",Квартира,None,Новострой,Панель,41.285358,69.222954,,4.0,4.0,Средний,60.0,2,24.02.2024,USD,65000,26.04.2024
6006,Uybor,"2 комнатная квартира в Яшнабадском районе, нов...",Квартира,None,Вторичка,Кирпич,41.288536,69.330872,,2.0,3.0,Средний,48.0,2,22.04.2024,USD,48000,26.04.2024
6007,Uybor,Продаётся квартира на Феруза 2. (Кадырова)!\n2...,Квартира,None,Новострой,Кирпич,41.354561,69.362778,,3.0,4.0,Капитальный,140.0,5,23.04.2024,USD,170000,26.04.2024


In [5]:
import pandas as pd
import os

# Assuming x is your variable and data is the data you want to store
name_of_file = "Uybor"
df = pd.DataFrame(new_df)

# Set the path to the Excels folder (assuming it is a sibling of the Notebooks folder)
excels_folder_path = os.path.join(os.path.dirname(os.getcwd()), "Excels")

# Check if the folder exists, if not, create it
if not os.path.exists(excels_folder_path):
    os.makedirs(excels_folder_path)

# Create a folder with the name_of_file only if it doesn't exist
file_folder_path = os.path.join(excels_folder_path, name_of_file)

if not os.path.exists(file_folder_path):
    os.makedirs(file_folder_path)

excel_file_name = os.path.join(file_folder_path, f"{name_of_file}.xlsx")

# Check if the file already exists
if os.path.exists(excel_file_name):
    # Read the existing Excel file into a DataFrame
    existing_df = pd.read_excel(excel_file_name)

    # Append the new data to the existing DataFrame
    updated_df = pd.concat([existing_df, df], ignore_index=True)

    # Check for duplicates in all columns
    duplicates_mask = updated_df.duplicated(keep=False)

    # Print the number of duplicates
    num_duplicates = duplicates_mask.sum()
    print(f"Number of duplicates after adding new data: {num_duplicates}")

    # If duplicates exist, remove them
    if any(duplicates_mask):
        updated_df = updated_df[~duplicates_mask]

    # Write the updated DataFrame back to the Excel file
    updated_df.to_excel(excel_file_name, index=False)

    print(f"Data added to existing Excel file '{excel_file_name}' after removing duplicates.")
else:
    # If the file doesn't exist, create a new Excel file with the data
    df.to_excel(excel_file_name, index=False)
    print(f"Excel file '{excel_file_name}' created with new data.")


Number of duplicates after adding new data: 2502
Data added to existing Excel file 'c:\Users\dilmu\OneDrive\Desktop\data_scrapping\Data Scrapping\Excels\Uybor\Uybor.xlsx' after removing duplicates.
